In [1]:
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np


from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PureSVDModel 
from rectools import Columns
from rectools.dataset import Dataset

from pathlib import Path
import typing as tp
from tqdm import tqdm

In [10]:
!pwd

/home/iuliiasolomennikova/!!!RecSysService/notebooks


In [12]:
DATA_PATH = Path("kion_train")

# LOAD DATA 

In [13]:
%%time
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

CPU times: user 4.58 s, sys: 756 ms, total: 5.34 s
Wall time: 5.66 s


# Preprocess

In [14]:
Columns.Datetime = 'last_watch_dt'

In [15]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [16]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [17]:
max_date = interactions[Columns.Datetime].max()

In [18]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [19]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [20]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [21]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [22]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Prepare features

## User features

In [23]:
users.isnull().sum()

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [24]:
users.fillna('Unknown', inplace=True)

In [25]:
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [26]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840184,529394,age_25_34,income_40_60,Ж,0
840186,80113,age_25_34,income_40_60,Ж,0
840188,312839,age_65_inf,income_60_90,Ж,0
840189,191349,age_45_54,income_40_60,М,1


In [27]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


In [28]:
user_features.query(f"id == 973171")

,id,value,feature
0,973171,М,sex
0,973171,age_25_34,age
0,973171,income_60_90,income


# Item features

In [29]:
items.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [30]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [31]:
items.head(3)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."


In [32]:
items.nunique()

item_id         14019
content_type        2
title           13454
title_orig       9724
release_year      104
genres           2559
countries         666
for_kids            2
age_rating          6
studios            38
directors        7414
actors          11830
description     13791
keywords        13583
dtype: int64

### Genre

In [33]:
# Explode genres to flatten table
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


### Content

In [34]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

In [35]:
content_feature

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


In [36]:
item_features = pd.concat((genre_feature, content_feature))

In [37]:
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


# Metrics

In [38]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

# Models

In [39]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
# N_FACTORS = (8, 16, 32, 64, 128, 256) # перебор параметров смотри ниже

In [40]:
%%time

dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

CPU times: user 2.83 s, sys: 80.9 ms, total: 2.91 s
Wall time: 2.98 s


In [41]:
# PureSVD matrix factorization model.
# set different number of latent factors to compute.

models = {
    'pure_svd': PureSVDModel(8,),
    'pure_svd_2': PureSVDModel(16,),
    'pure_svd_3': PureSVDModel(32,),
    'pure_svd_4': PureSVDModel(64,),
    'pure_svd_5': PureSVDModel(128,),
    'pure_svd_6': PureSVDModel(256,)
}

In [42]:
models

{'pure_svd': <rectools.models.pure_svd.PureSVDModel at 0x7fe3020f08e0>,
 'pure_svd_2': <rectools.models.pure_svd.PureSVDModel at 0x7fe3020f0dc0>,
 'pure_svd_3': <rectools.models.pure_svd.PureSVDModel at 0x7fe3020f0be0>,
 'pure_svd_4': <rectools.models.pure_svd.PureSVDModel at 0x7fe3020f09a0>,
 'pure_svd_5': <rectools.models.pure_svd.PureSVDModel at 0x7fe3020f0c40>,
 'pure_svd_6': <rectools.models.pure_svd.PureSVDModel at 0x7fe3020f0d00>}

In [43]:
%%time

results = []

for model_name, model in models.items():
    print(f"Fitting model {model_name}...")
    model_quality = {'model': model_name}

    model.fit(dataset)
    recos = model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality.update(metric_values)
    results.append(model_quality)

Fitting model pure_svd...
Fitting model pure_svd_2...
Fitting model pure_svd_3...
Fitting model pure_svd_4...
Fitting model pure_svd_5...
Fitting model pure_svd_6...
CPU times: user 1h 14min 30s, sys: 2min 22s, total: 1h 16min 52s
Wall time: 2h 58min 32s


In [44]:
df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [45]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

model,pure_svd,pure_svd_2,pure_svd_3,pure_svd_4,pure_svd_5,pure_svd_6
Precision@1,0.026315,0.024925,0.029201,0.022456,0.023828,0.018393
Recall@1,0.012137,0.010515,0.011992,0.008966,0.009600,0.007101
Precision@2,0.022695,0.024514,0.027014,0.020256,0.020641,0.016357
Recall@2,0.020639,0.020108,0.022427,0.015870,0.016141,0.012381
Precision@3,0.021659,0.025259,0.025554,0.018520,0.018476,0.014413
Recall@3,0.029493,0.031904,0.031759,0.021282,0.021278,0.015942
Precision@4,0.020967,0.024248,0.023883,0.017103,0.016922,0.013514
Recall@4,0.038796,0.040916,0.039387,0.025615,0.025704,0.019673
Precision@5,0.019742,0.022714,0.022651,0.016409,0.015927,0.012645
Recall@5,0.045579,0.047574,0.046422,0.030485,0.029812,0.022814


In [46]:
# best model

K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
N_FACTORS = (16,) 

dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

models = {
    'pure_svd': PureSVDModel(16,)
}

results = []
for model_name, model in models.items():
    print(f"Fitting model {model_name}...")
    model_quality = {'model': model_name}

    model.fit(dataset)
    recos = model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality.update(metric_values)
    results.append(model_quality)

Fitting model pure_svd...


In [47]:
recos

,user_id,item_id,score,rank
0,203219,11237,0.351069,1
1,203219,4151,0.183803,2
2,203219,7417,0.130605,3
3,203219,142,0.109011,4
4,203219,2657,0.105311,5
...,...,...,...,...
1129765,857162,1449,0.069983,6
1129766,857162,6402,0.057338,7
1129767,857162,14526,0.048205,8
1129768,857162,12192,0.047171,9


In [48]:
# Save recomendations

recos.to_csv('/home/iuliiasolomennikova/!!!RecSysService/notebooks/offline_recommend_df/svd', encoding='utf-8', index=False)

In [49]:
def create_dates(start, end, n=15):
    ndays = (end - start).days + 1
    range_ = np.random.randint(0, ndays, n)
    res_dates = pd.to_timedelta(range_, unit='D') + start
    return res_dates

# TEST USERS  AVATARS
искусственных пользователей

In [50]:
N_watched = 15

# User, who watch only movies for child for example

random_items = np.random.choice(
    np.array(items[items.for_kids == 1].item_id),
    N_watched,
    replace=False,
)

user_id = np.full(
  shape=N_watched,
  fill_value=-1,
  dtype=np.int
)

last_watch_dt = create_dates(pd.to_datetime(interactions[Columns.Datetime].min()), pd.to_datetime((interactions[Columns.Datetime].max())), N_watched)
total_dur = np.random.uniform(7000, 15000, size=N_watched).astype('int')
watched_pct = np.random.uniform(30, 100, size=N_watched).astype('int').astype('float64')
weight = np.full(
  shape=N_watched,
  fill_value=3,
  dtype=np.int
)

test_user_1 = pd.DataFrame(
    {'user_id': user_id,
     'item_id': random_items,
     'last_watch_dt': last_watch_dt,
     'total_dur': total_dur,
     'watched_pct': watched_pct,
     'weight': weight,
    })


# User, who watch only one genre's movies - боевики
random_items = np.random.choice(
    np.array(items[Columns.Item][items['genres'].apply(lambda x: 'боевики' in x)].unique()),
    N_watched,
    replace=False,
)

user_id = np.full(
  shape=N_watched,
  fill_value=-2,
  dtype=np.int
)
last_watch_dt = create_dates(pd.to_datetime(interactions[Columns.Datetime].min()), pd.to_datetime((interactions[Columns.Datetime].max())), N_watched)
watched_pct = np.random.uniform(30, 100, size=N_watched).astype('int').astype('float64')
weight = np.full(
  shape=N_watched,
  fill_value=3,
  dtype=np.int
)
test_user_2 = pd.DataFrame(
    {'user_id': user_id,
     'item_id': random_items,
     'last_watch_dt': last_watch_dt,
     'total_dur': total_dur,
     'watched_pct': watched_pct,
     'weight': weight,
    })

# User, who watch only russian movies with age_rating > 18
random_items = np.random.choice(
    np.array(items[(items.countries == 'Россия') & (items.age_rating == 18.0)].item_id.unique()),
    N_watched,
    replace=False,
)
user_id = np.full(
  shape=N_watched,
  fill_value=-3,
  dtype=np.int
)
last_watch_dt = create_dates(pd.to_datetime(interactions[Columns.Datetime].min()), pd.to_datetime((interactions[Columns.Datetime].max())), N_watched)
watched_pct = np.random.uniform(30, 100, size=N_watched).astype('int').astype('float64')
weight = np.full(
  shape=N_watched,
  fill_value=3,
  dtype=np.int
)
test_user_3 = pd.DataFrame(
    {'user_id': user_id,
     'item_id': random_items,
     'last_watch_dt': last_watch_dt,
     'total_dur': total_dur,
     'watched_pct': watched_pct,
     'weight': weight,
    })

In [51]:
# best model based on pur calculations

K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
N_FACTORS = (16,) 

dataset = Dataset.construct(
    interactions_df=pd.concat([train, test_user_1, test_user_2, test_user_3]),
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

#the best latent factors
models = {
    'pure_svd': PureSVDModel(16,)
}

results = []
for model_name, model in models.items():
    print(f"Fitting model {model_name}...")
    model_quality = {'model': model_name}

    model.fit(dataset)
    recos = model.recommend(
        users=np.append(test[Columns.User].unique(), np.array([-1, -2, -3])),
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality.update(metric_values)
    results.append(model_quality)

Fitting model pure_svd...


In [52]:
items[items.item_id.isin(recos[recos.user_id == -1].item_id)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre
158,13915,film,Вперёд,Onward,2020.0,"для детей, приключения, семейное, фэнтези, ком...",США,NaN,6.0,NaN,Дэн Скэнлон,"Том Холланд, Крис Пратт, Джулия Луис-Дрейфус, ...","Когда-то давно в сказочном мире царила магия, ...","эльфы, мир фантазий, эльф, главный герой подро...","[для детей, приключения, семейное, фэнтези, ко..."
1274,5411,film,Монстры на каникулах 3: Море зовёт,Hotel Transylvania 3: Summer Vacation,2018.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,6.0,NaN,Генндий Тартаковский,"Энди Сэмберг, Селена Гомес, Кевин Джеймс, Стив...",На этот раз Дракула вместе со своими друзьями ...,"третья часть, круизное судно, персонаж Дракула...","[мультфильм, фэнтези, приключения, комедии]"
1554,16270,film,Тайна Коко,Coco,2017.0,"мультфильм, фэнтези, приключения",США,NaN,12.0,NaN,"Ли Анкрич, Эдриан Молина","Энтони Гонсалес, Гаэль Гарсиа Берналь, Бенджам...",Мексиканский юноша Мигель живёт в семье сапожн...,"Мексика, гитара, музыкант, скелет, музыка, заг...","[мультфильм, фэнтези, приключения]"
1849,7571,film,100% волк,100% Wolf,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",NaN,6.0,NaN,Алекс Стадерманн,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...",Наследник семьи оборотней Фредди Люпин отчаянн...,"пудель, подростковая тревога, оборотень, приня...","[мультфильм, приключения, семейное, фэнтези, к..."
3951,3182,film,Ральф против Интернета,Ralph Breaks the Internet,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,NaN,6.0,NaN,"Рич Мур, Фил Джонстон","Джон Си Райли, Сара Силверман, Галь Гадот, Тар...",На этот раз Ральф и Ванилопа фон Кекс выйдут з...,"видеоигра, мультфильм, продолжение, интернет, ...","[мультфильм, приключения, фантастика, семейное..."
9698,16166,film,Зверополис,Zootopia,2016.0,"приключения, мультфильм, детективы, комедии",США,NaN,6.0,NaN,"Байрон Ховард, Рич Мур, Джаред Буш","Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...",Добро пожаловать в Зверополис — современный го...,"аллегория, лев, бегемот, лиса, слон, овца, бел...","[приключения, мультфильм, детективы, комедии]"
9921,10761,film,Моана,Moana,2016.0,"мультфильм, фэнтези, мюзиклы",США,NaN,6.0,NaN,"Рон Клементс, Джон Маскер, Дон Холл","Аулии Кравальо, Дуэйн Джонсон, Рэйчел Хаус, Те...","Бесстрашная Моана, дочь вождя маленького племе...","океан, парусник, море, мифология, остров, мюзи...","[мультфильм, фэнтези, мюзиклы]"
11254,11749,film,Суперсемейка 2,Incredibles 2,2018.0,"фантастика, мультфильм, приключения",США,NaN,6.0,NaN,Брэд Бёрд,"Крэйг Т. Нельсон, Холли Хантер, Сара Вауэлл, Х...",После событий первой части суперсемейка пользу...,"семейная пара, продолжение, супергерой, дети, ...","[фантастика, мультфильм, приключения]"
12620,13243,film,Головоломка,Inside Out,2015.0,"фантастика, мультфильм, комедии",США,NaN,6.0,NaN,"Пит Доктер, Роналдо Дель Кармен","Эми Полер, Филлис Смит, Ричард Кайнд, Билл Хей...",11-летняя девочка Райли переезжает с семьёй из...,"мечта, мультфильм, воображаемый друг, начальна...","[фантастика, мультфильм, комедии]"
15352,7582,film,Холодное сердце II,Frozen II,2019.0,"фэнтези, мультфильм, музыкальные",США,NaN,6.0,NaN,"Крис Бак, Дженнифер Ли","Идина Мензел, Кристен Белл, Джонатан Грофф, Дж...","Анна, Эльза, Кристоф, его верный олень Свен и ...","королева, магия, королевство, плотина, дух, же...","[фэнтези, мультфильм, музыкальные]"


In [53]:
items[items.item_id.isin(recos[recos.user_id == -2].item_id)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre
6839,10464,film,Вирус страха,Before the fire,2020.0,"драмы, триллеры",США,NaN,16.0,NaN,Чарли Бюхлер,"Дженна Линг, Джексон Дэвис, Райан Виджилант, Ч...",Из-за начавшейся пандемии известная актриса вы...,"выживальщик, нелегальная милиция, 2020, соедин...","[драмы, триллеры]"
7734,4495,film,Пальмира,Palmyra,2020.0,драмы,Россия,NaN,16.0,NaN,Иван Болотников,"Геза Морчани, Екатерина Крамаренко, Даниэла Ст...",Бывший военный врач Артур ведет тихую жизнь в ...,"2020, россия, пальмира",[драмы]
8291,12995,film,Восемь сотен,Ba bai,2020.0,"боевики, драмы, историческое, военные",Китай,NaN,18.0,NaN,Гуань Ху,"Ван Цяньюань, Цзян У, Хуан Чжичжун, Чжан Цзюнъ...","26 октября 1937 года, Японо-китайская война. 4...","осада, китайско-японская война, 2020, китай, в...","[боевики, драмы, историческое, военные]"
9477,14741,film,Цвет из иных миров,Colour out of space,2020.0,"фантастика, ужасы","США, Малайзия, Португалия",NaN,16.0,NaN,Ричард Стэнли,"Николас Кейдж, Джоэли Ричардсон, Мадлен Артур,...",Экранизация рассказа Говарда Лавкрафта про упа...,"мутация, хижина, сарай, отшельник, ферма, мете...","[фантастика, ужасы]"
9558,7793,film,Радиовспышка,Radioflash,2019.0,"боевики, драмы, фантастика, триллеры",США,NaN,16.0,NaN,Бен Макферсон,"Брайтон Шарбино, Доминик Монахэн, Уилл Пэттон,...","Риз с легкостью проходит виртуальные квесты, н...","2019, соединенные штаты, радиовспышка","[боевики, драмы, фантастика, триллеры]"
9938,14431,film,Приворот. Чёрное венчание,Privorot. Chernoe venchanie,2021.0,"ужасы, триллеры, мелодрамы",Россия,NaN,16.0,NaN,Святослав Подгаевский,"Яна Енжаева, Константин Белошапка, Илья Ермоло...",Отечественный хоррор с любовной историей. Женю...,", 2021, россия, приворот, чёрное, венчание","[ужасы, триллеры, мелодрамы]"
10087,7829,film,Поступь хаоса,Chaos Walking,2021.0,"боевики, фантастика, фэнтези, приключения","США, Канада, Люксембург",NaN,16.0,NaN,Даг Лайман,"Том Холланд, Дэйзи Ридли, Демиан Бичир, Дэвид ...",2257 год. Родина Тодда Хьюитта — колонизирован...,"по роману или книге, постапокалиптическое буду...","[боевики, фантастика, фэнтези, приключения]"
14383,4457,film,2067: Петля времени,2067,2020.0,"драмы, фантастика",Австралия,NaN,16.0,NaN,Сет Ларни,"Аарон Гленнэйн, Коди Смит-МакФи, Райан Квантен...",2067 год. Люди окончательно загубили экологию ...,"изменение климата, путешествие во времени, выр...","[драмы, фантастика]"
14958,7102,film,Дочь волка,Daughter of the Wolf,2019.0,"боевики, триллеры",Канада,NaN,18.0,NaN,Дэвид Хэкл,"Джина Карано, Брендан Фер, Ричард Дрейфусс, Ст...",Клэр возвращается домой из поездки по Ближнему...,"2019, канада, дочь, волка","[боевики, триллеры]"
15438,14703,film,Денежная игла,Body brokers,2021.0,триллеры,США,NaN,18.0,NaN,Джон Своб,"Фрэнк Грилло, Майкл Кеннет Уильямс, Джессика Р...",Основанная на реальных событиях драма с Джеком...,"2021, соединенные штаты, денежная, игла",[триллеры]


In [54]:
items[items.item_id.isin(recos[recos.user_id == -3].item_id)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre
6425,11237,film,День города,NaN,2021.0,комедии,Россия,NaN,16.0,NaN,Алексей Харитонов,"Катерина Шпица, Антон Филипенко, Павел Ворожцо...",Эта история случилась в провинциальном городке...,"2021, россия, день, города",[комедии]
8291,12995,film,Восемь сотен,Ba bai,2020.0,"боевики, драмы, историческое, военные",Китай,NaN,18.0,NaN,Гуань Ху,"Ван Цяньюань, Цзян У, Хуан Чжичжун, Чжан Цзюнъ...","26 октября 1937 года, Японо-китайская война. 4...","осада, китайско-японская война, 2020, китай, в...","[боевики, драмы, историческое, военные]"
9253,1844,film,Аферистка,I Care A Lot,2020.0,"триллеры, комедии",Великобритания,NaN,18.0,NaN,Джей Блейксон,"Розамунд Пайк, Питер Динклэйдж, Эйса Гонсалес,...",Марла руководствуется принципом «побеждает сил...,"опекун, аферист, чёрная комедия, мошенник, 202...","[триллеры, комедии]"
9477,14741,film,Цвет из иных миров,Colour out of space,2020.0,"фантастика, ужасы","США, Малайзия, Португалия",NaN,16.0,NaN,Ричард Стэнли,"Николас Кейдж, Джоэли Ричардсон, Мадлен Артур,...",Экранизация рассказа Говарда Лавкрафта про упа...,"мутация, хижина, сарай, отшельник, ферма, мете...","[фантастика, ужасы]"
9938,14431,film,Приворот. Чёрное венчание,Privorot. Chernoe venchanie,2021.0,"ужасы, триллеры, мелодрамы",Россия,NaN,16.0,NaN,Святослав Подгаевский,"Яна Енжаева, Константин Белошапка, Илья Ермоло...",Отечественный хоррор с любовной историей. Женю...,", 2021, россия, приворот, чёрное, венчание","[ужасы, триллеры, мелодрамы]"
10087,7829,film,Поступь хаоса,Chaos Walking,2021.0,"боевики, фантастика, фэнтези, приключения","США, Канада, Люксембург",NaN,16.0,NaN,Даг Лайман,"Том Холланд, Дэйзи Ридли, Демиан Бичир, Дэвид ...",2257 год. Родина Тодда Хьюитта — колонизирован...,"по роману или книге, постапокалиптическое буду...","[боевики, фантастика, фэнтези, приключения]"
13615,2657,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Илья Куликов,"Александр Hовиков, Валентина Ляпина, Никита Па...",Смотри:- как кино- как сериалПодростковый псих...,"подслушано, подслушано в контакте, социальная ...","[драмы, триллеры]"
14383,4457,film,2067: Петля времени,2067,2020.0,"драмы, фантастика",Австралия,NaN,16.0,NaN,Сет Ларни,"Аарон Гленнэйн, Коди Смит-МакФи, Райан Квантен...",2067 год. Люди окончательно загубили экологию ...,"изменение климата, путешествие во времени, выр...","[драмы, фантастика]"
14958,7102,film,Дочь волка,Daughter of the Wolf,2019.0,"боевики, триллеры",Канада,NaN,18.0,NaN,Дэвид Хэкл,"Джина Карано, Брендан Фер, Ричард Дрейфусс, Ст...",Клэр возвращается домой из поездки по Ближнему...,"2019, канада, дочь, волка","[боевики, триллеры]"
15234,142,film,Маша,Masha,2020.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Анастасия Пальчикова,"Максим Суханов, Аня Чиповская, Полина Гухман, ...",13-летняя Маша растёт между боксёрским рингом ...,"Фильм Маша, Маша фильм 2021, Смотреть фильм Ма...","[драмы, триллеры]"
